# SPAM FILTER

In [57]:
# Import modules

In [2]:
import os
import sys
import pandas as pd
import email
import re
import nltk
import numpy as np
import sklearn


# Functions 

## Read data using file crawler

In [2]:
# Mails staan in folders, subfolders. FileCrawler loopt door de folders en subfolders heb en opent de mails.

In [3]:
# File crawler om data in te lezen
def FileCrawler(root, variable):
    for root, dirs, files in os.walk(root):
        for f in files: 
            with open(os.path.join(root,f)) as content:
                file_content = content.read()
                variable.append(file_content) 
    return

## Mails uit elkaar trekken

In [4]:
# mails bestaan uit mail-informatie en het daadwerkelijke mailtje (dus from, to, etc.).
# SpamParser haalt de informatie los en geeft de "from, to, en body" als losse informatie.

In [5]:
def SpamParser(data):
    b = email.message_from_string(data)
    mail_from = b['from']
    mail_to = b['to']
    mail_body = []
    if b.is_multipart():
        for part in b.walk():
            if part.get_content_type() == 'text/plain':
                mail_body = part.get_payload()
            elif part.get_content_type() == 'text/html':
                mail_body = part.get_payload()
    else:
        mail_body = b.get_payload()
    return {'from':mail_from, 'to': mail_to,'body': mail_body}


## Line breaks verwijderen uit mail

In [6]:
def RemoveLineBreaks(mail):
    cleaned = re.sub("\n", " ", mail)
    cleaned = re.sub("\t", " ", cleaned)
    return cleaned

## Mails HTML to text

In [7]:
# De spam mail bodies staan in html
# Clean_html: functie om html tags uit mailtje te halen. Werkt niet 100% (soms blijft er een enkele code staan) 
# maar wel beter dan html2text (encoding errors) en BeautifulSoup (die laat soms html staan zonder het te zeggen)

In [8]:
def clean_html(file):
    # First remove inline JavaScript/CSS:
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?()", "", file)
    # Then remove html comments. 
    cleaned = re.sub(r"(?s)[\n]?", "", cleaned)
    # Next remove the remaining tags:
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
    # Finally deal with whitespace
    cleaned = cleaned.replace('&nbsp', ' ')
    cleaned = re.sub(r" ", " ", cleaned)
    cleaned = re.sub(r"^$", "", cleaned)
    cleaned = re.sub("''|,", "", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    cleaned = re.sub("\t", " ", cleaned)
    cleaned = re.sub("\n", " ", cleaned)
    return cleaned

In [9]:
    # Functie om de mail body om te zetten in een string woorden. 
    # Niet-letters worden verwijderd
    # alles is lower-case, opgesplitst in losse woorden
    # stop-woorden zijn verwijderd
    # overgebleven woorden weer omgezet in enkele string

In [10]:
def mail_to_words( raw_mailbody ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    #
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", raw_mailbody) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

# Clean and preprocess data

## Data inlezen

In [11]:
# HAM-data inlezen
root = "C:\Users\HOosterhuis\Documents\BDR\SPAMFILTER\HAM"
ham_data = []
FileCrawler(root, ham_data)

In [12]:
ham_data[12]

"Message-ID: <17309347.1075855814379.JavaMail.evans@thyme>\nDate: Thu, 17 Aug 2000 00:48:00 -0700 (PDT)\nFrom: ted.bland@enron.com\nTo: david.oxley@enron.com, sally.beck@enron.com, raymond.bowen@enron.com, \n\tchristopher.calger@enron.com, tim.belden@enron.com, \n\tmax.yzaguirre@enron.com, rob.milnthorp@enron.com, \n\tjanet.dietrich@enron.com, george.mccormick@enron.com, \n\tjeff.donahue@enron.com, heather.kendall@enron.com, \n\tgary.hickerson@enron.com, elsa.piekielniak@enron.com, \n\tjesus.melendrez@enron.com, rodney.malcolm@enron.com, \n\tkevin.mcgowan@enron.com, jere.overdyke@enron.com, \n\tedward.baughman@enron.com, tom.shelton@enron.com, \n\tjean.mrha@enron.com, bryan.burnett@enron.com, \n\tvince.kaminski@enron.com, mike.mcconnell@enron.com\nSubject: Initial meeting of the ENA Analyst and Associate Roundtable August\n 18, 2000\nCc: mark.frevert@enron.com, david.delainey@enron.com, john.lavorato@enron.com, \n\ttim.o'rourke@enron.com\nMime-Version: 1.0\nContent-Type: text/plain; ch

In [13]:
# SPAM-data inlezen
root = "C:\Users\HOosterhuis\Documents\BDR\SPAMFILTER\SPAM"
spam_data = []
FileCrawler(root, spam_data)

In [14]:
spam_data[15]

'Return-Path: <rollinsbc@to-pornostarz.de>\nDelivered-To: cvs@bruce-guenter.dyndns.org\nReceived: (qmail 9893 invoked from network); 31 Jul 2004 05:52:48 -0000\nReceived: from localhost (localhost [127.0.0.1])\n  by bruce-guenter.dyndns.org ([192.168.1.3]); 31 Jul 2004 05:52:48 -0000\nReceived: from zak.futurequest.net ([127.0.0.1])\n  by localhost ([127.0.0.1])\n  with SMTP via TCP; 31 Jul 2004 05:52:48 -0000\nReceived: (qmail 5447 invoked from network); 31 Jul 2004 05:52:47 -0000\nReceived: from io.dk (unknown [221.138.100.167])\n  by zak.futurequest.net ([69.5.6.152])\n  with SMTP via TCP; 31 Jul 2004 05:52:46 -0000\nReceived: from 238.196.223.0 by smtp.to-pornostarz.de;\n\tSat, 31 Jul 2004 05:53:06 +0000\nMessage-ID: <acca01c476c2$cdaa0f98$4cd3b42c@io.dk>\nFrom: rollinsbc@to-pornostarz.de\nTo: cvs@bruce-guenter.dyndns.org\nSubject: Make $472\nDate: Sat, 31 Jul 2004 11:52:59 +0600\nMIME-Version: 1.0\nContent-Type: text/plain;\n\tcharset="iso-8859-1"\nContent-Transfer-Encoding: 8bit\

## Ham-email omzetten

In [15]:
# Ham-mails: from, to en body extracten

In [16]:
ham_data_mail = [SpamParser(d) for d in ham_data]

In [17]:
ham_data_from = [x['from'] for  x in ham_data_mail]
ham_data_to = [x['to'] for  x in ham_data_mail]
ham_data_body = [x['body'] for  x in ham_data_mail]

In [18]:
ham_data_text = [RemoveLineBreaks(x) for x in ham_data_body]

In [19]:
ham_data_text[10]

'URGENT  The Associate and Analyst Recruiting Department will be conducting a number  of two hour workshops to review our recruiting and interview process for the  fall on-campus recruiting effort.  Critical information regarding our  on-campus interview process, revised evaluation forms and program structure  will be reviewed during these two hours sessions.  It is mandatory that all team members attend these workshops.  All team  members must attend in order to articulate and demonstrate the Enron  recruiting process.  Knowing how busy schedules are, we have made  arrangements to present these workshops in two hours sessions for a total of   40 workshops that will run during the last week of August, through the month  of September and end at mid October.  Listed below are the dates, location and times for each session.  Please  select a date and time and e-mail this information to my assistant, Dolores  Muzzy.  We can accommodate 25 participants at a time.  Dolores will take  dates a

## Spam email omzetten 

In [20]:
spam_data_mail = [SpamParser(d) for d in spam_data]

In [21]:
# Spam_data onderdelen los in lists opslaan, zodat verder gewerkt kan worden aan de (html-)body alleen

In [22]:
spam_data_from = [x['from'] for  x in spam_data_mail]
spam_data_to = [x['to'] for  x in spam_data_mail]
spam_data_body = [x['body'] for  x in spam_data_mail]

In [23]:
# Checken hoeveel mails zijn overgebleven

In [24]:
len(spam_data_body)

32959

In [25]:
# Lege mails er uit halen (zijn er maar een paar)

In [26]:
spam_data_body = filter(None, spam_data_body)

In [27]:
len(spam_data_body)

32848

## Spam data in HTML: omzetten naar ASCII

In [28]:
# Mail body staat nog in HTML

In [29]:
spam_data_body[14]

'<html>\n<font color=\n\n#e5e6f6>Carolyn regretful watchfully Procrustes godly</font><br>\nSummer 2004 was too hot for the software manufacturers.<br>\nNo wonder! As the prices were reduced in 3-4 times.<br>\nThis was caused by the software glut on the world market.<br>\nOn the other hand the user who were not able or just had<br>\nno time to update their software now have the possibility<br>\nto do this almost free of charge.<br>\n<br>\n<b>\n<i>Read the whole article:</i><br>\n<a hrefrockerhref=http://ordinal.com href=\n\n"http://softbox24.info/index.php?s=0288">Year 2004. Sotware prices fall down.</a><br>\n<font size=-1>\xc2\xa9 Peter Lemelman</font>\n</font>\n</b>\n<font color=\n\n#e4e6e4>onerous reclaimers remunerate Lounsbury dictate</font><br>\n<font color=\n\n#fbfdf1>costed continued snooping digression Rhine</font><br>\n<font color=\n\n#f2f5f5>inseminate tilts instructs rejoice switchman</font><br>\n<font color=\n\n#e3e8e7>stomaching hurtling Brent gunners tortoises</font><br>\

In [30]:
# omzetten naar text met clean_html

In [31]:
spam_body_text = [clean_html(x) for x in spam_data_body]

In [32]:
spam_body_text[5]

"      Cheap softtwares for you all are Original Genuine!  Major titles from MICR0S0FT and AD0BE for Rock Bottom prriice Great Bargaain Sa1e! Variety discoount softtwares at wholesale chaeap pricing!   Micros0ft Wind0ws XP PR0fessional -  my price: $50 ;  normal : $299.00 ;  you saave $249.00  Ad0be Ph0toshop CS V 8.O PC  -  my price: $80 ;  normal : $609.99 ;  you save $529.99  Micros0ft 0ffice XP PR0fessional -  my price: $100 ;  normal : $ 499.95;  you saave $399.95  Ad0be Acrobaat V 6.O Professional PC -  my price: $100 ;  normal : $449.95 ;  you saave $349.95  Micros0ft 0ffice 2OO3 Professional -  my price: $80 ;  normal : $499.95 ;  you saave $419.95  N0rton Antivirus 2OO4 Professional -  my price: $15 ;  normal : $69.95 ;  you saave $54.95  CorelDraw Graphics Suite V 12 PC -  my price: $100 ;  normal : $349.95 ;  you saave $249.95  Ad0be Pagemaker V 7.O PC  -  my price: $80 ;  normal : $599.95 ;  you saave $519.95  We do have full range softwares -- Macr0media Mc-Afeee Ad0bee Co

# Create dataframe

In [33]:
raw_spam = pd.DataFrame(spam_body_text, columns=["text"])

In [34]:
raw_spam['Spam'] = pd.Series([1 for x in range(len(raw_spam.index))], index=raw_spam.index)

In [35]:
raw_spam

,text,Spam
0,shakedown NEWSLETTER stop NEWS bushel ISSUE ...,1
1,Your [m]ortgage application was approved. You...,1
2,TOP quality software: Special Offer #1: Wind...,1
3,Promotional Newsletter ...,1
4,- Flyer A:hover {color: #FF0000} ...,1
5,Cheap softtwares for you all are Origina...,1
6,Dear Friend REQUEST FOR URGENT BUSINESS ASSIST...,1
7,I need you ; ; ; ; ; ; ...,1
8,basalt deducible oaken fafnir hamlin uncle ...,1
9,broadside outlay spores devoted vexing kicke...,1


In [36]:
raw_ham = pd.DataFrame(ham_data_text, columns=["text"])

In [37]:
raw_ham['Spam'] = pd.Series([0 for x in range(len(raw_ham.index))], index=raw_ham.index)

In [38]:
raw_ham

,text,Spam
0,"Guys, attached you will find a final cut on th...",0
1,I am still in need of the information regardin...,0
2,Attached is a file containing all cost centers...,0
3,"Sarah, Below is our high level explanation of ...",0
4,Sally - I will check into the meaning of this ...,0
5,Please note that earnings and headcount templa...,0
6,I wanted to provide an update on our progress ...,0
7,"Rob, Thank you for bringing this to my attent...",0
8,"Susan, I understand that Isabel Recindes, Drew...",0
9,We are making progress but we still need 12 ad...,0


In [39]:
frames = [raw_spam,raw_ham]
mail_df = pd.concat(frames)

In [40]:
mail_df

,text,Spam
0,shakedown NEWSLETTER stop NEWS bushel ISSUE ...,1
1,Your [m]ortgage application was approved. You...,1
2,TOP quality software: Special Offer #1: Wind...,1
3,Promotional Newsletter ...,1
4,- Flyer A:hover {color: #FF0000} ...,1
5,Cheap softtwares for you all are Origina...,1
6,Dear Friend REQUEST FOR URGENT BUSINESS ASSIST...,1
7,I need you ; ; ; ; ; ; ...,1
8,basalt deducible oaken fafnir hamlin uncle ...,1
9,broadside outlay spores devoted vexing kicke...,1


In [50]:
mail_df = mail_df.reset_index(drop=True)

In [51]:
mail_df

,text,Spam
0,shakedown NEWSLETTER stop NEWS bushel ISSUE ...,1
1,Your [m]ortgage application was approved. You...,1
2,TOP quality software: Special Offer #1: Wind...,1
3,Promotional Newsletter ...,1
4,- Flyer A:hover {color: #FF0000} ...,1
5,Cheap softtwares for you all are Origina...,1
6,Dear Friend REQUEST FOR URGENT BUSINESS ASSIST...,1
7,I need you ; ; ; ; ; ; ...,1
8,basalt deducible oaken fafnir hamlin uncle ...,1
9,broadside outlay spores devoted vexing kicke...,1


# Divide data into training (60%), cross-validation (20%), and test set (20%) 

In [41]:
# Dataframe opdelen in een trainset, crossvalidation set, en test set. 
# De eerste is om de modellen te trainen, 
# de tweede is om de getrainde modellen te vergelijken en het beste model te selecteren. 
# De test set is om te kijken hoe het uiteindelijk gekozen model het doet op nieuwe data.

In [52]:
train, validate, test = np.split(mail_df.sample(frac=1), [int(.6*len(mail_df)), int(.8*len(mail_df))])

In [53]:
# rowmames aanpassen; 0, 1, 2, etc.

In [54]:
num_train_mails = len(train["text"])
train.index = range(num_train_mails)

In [55]:
train

,text,Spam
0,"Hi Vince, ? I'm sorry you weren't around in Sy...",0
1,Denis Terrell ...,1
2,It is amazing and yet not surprising how much ...,0
3,I know what you're going through inside is ...,1
4,CIBRES.COM ; Remove ;148827 Subsc...,1
5,All Your Meds Here We are you...,1
6,This is a multi-part message in MIME for...,1
7,Corporate image can say a lot of things =...,1
8,fireproof stroboscopic cattleman...,1
9,Heyz Hey it's me Megan... my husband is ...,1


# Extract features

In [46]:
# Veelgebruikte woorden verwijderen uit de trainingsdata
from nltk.corpus import stopwords

In [56]:
clean_train_mails = []

In [57]:
for i in xrange( 0, num_train_mails ): 
    clean_train_mails.append( mail_to_words( train["text"][i] ))

In [58]:
clean_train_mails[0]

'hi vince sorry around sydney week missed good book launch party john martin organised us paul made short speech relayed great comments said came thanks much please find attached next eprm article really tided fully end yet wanted send weekend case got chance look easter break robin thursday next week best regards chris eprm fwd curve simulation doc'

## Bag of words 

In [ ]:
# 5000 meest gebruikte woorden (die informatief zijn) gebruiken als features.
# In een matrix zetten met 0/1 om aan te geven of het woord voorkomt in een bepaalde mail
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
# "CountVectorizer": scikit-learn's bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_mails)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [ ]:
# Aantal mails, aantal woorden (5000)

In [67]:
print train_data_features.shape

(31162L, 5000L)


In [ ]:
# De geselecteerde woorden bekijken

In [68]:
vocab = vectorizer.get_feature_names()
print vocab[0:50]

[u'aa', u'aacute', u'ab', u'abi', u'ability', u'able', u'abroad', u'absence', u'absolute', u'absolutely', u'abuse', u'ac', u'accept', u'acceptable', u'acceptance', u'accepted', u'accepting', u'accepts', u'access', u'accessories', u'accident', u'accomplish', u'accordance', u'according', u'accordingly', u'account', u'accounting', u'accounts', u'accuracy', u'accurate', u'ace', u'aced', u'achieve', u'achieved', u'acni', u'acquire', u'acquired', u'acquiring', u'acquisition', u'acquisitions', u'acres', u'acrobat', u'across', u'act', u'action', u'actions', u'active', u'activities', u'activity', u'acts']


In [73]:
# Woord count is niet helemaal eerlijk, want langere mails meer woorden, dus daar rekening mee houden

# Train model

In [ ]:
# Random forest schatten (100 trees)
from sklearn.ensemble import RandomForestClassifier

In [154]:
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_data_features, train["Spam"] )


In [ ]:
# Naieve Bayes schatten
from sklearn.naive_bayes import MultinomialNB

In [76]:
clf = MultinomialNB().fit(train_data_features, train["Spam"])

# Cross-validate model

In [ ]:
# Kijken hoe goed de modellen het doen

In [ ]:
# Eerst data bewerken zodat het in het model gebruikt kan worden (zelfde bewerkingen als traindata)

In [79]:
clean_validate_mails = []
num_validate_mails = len(validate["text"])
validate.index = range(num_validate_mails)

for i in xrange( 0, num_validate_mails ): 
    clean_validate_mails.append( mail_to_words( validate["text"][i] ))

In [ ]:
# Bag of words maken (maar niet fitten!)

In [80]:
validate_data_features = vectorizer.transform(clean_validate_mails)
validate_data_features = validate_data_features.toarray()

In [82]:
print validate_data_features.shape

(10387L, 5000L)


## Random forest

In [ ]:
## 100 trees

In [155]:
result_forest = forest.predict(validate_data_features)

In [156]:
# Classification error, Type I error, Type II error

In [157]:
np.mean(result_forest==validate["Spam"])*100

98.594396842206606

In [158]:
100 - np.mean(result_forest==validate["Spam"])*100

1.4056031577933936

In [100]:
# 98.6% goed voorspeld of de cross-validation mails Spam zijn
# 1.4% is fout

In [ ]:
# Type I fout (dus het is eigenlijk geen Spam, maar je hebt het wel zo aangemerkt)
# Type II fout (dus het is eigenlijk wel Spam, maar het wordt niet zo aangemerkt)

In [93]:
np.mean(result_forest>validate["Spam"])*100

0.72205641667468956

In [94]:
np.mean(result_forest<validate["Spam"])*100

0.63540964667372679

In [ ]:
# Dus 0.72% Type I fouten gemaakt
# Dus 0.64% Type II fouten gemaakt

In [ ]:
# Ongeveer evenveel Type I en Type II fouten

## Naieve Bayes

In [96]:
result_clf = clf.predict(validate_data_features)

In [101]:
np.mean(result_clf==validate["Spam"])*100

97.323577548859149

In [102]:
100 - np.mean(result_clf==validate["Spam"])*100

2.6764224511408514

In [ ]:
# Dus 97.3% is goed voorspeld, 2.7% fout voorspeld

In [ ]:
# Type I fout (dus het is eigenlijk geen Spam, maar je hebt het wel zo aangemerkt)
# Type II fout (dus het is eigenlijk wel Spam, maar het wordt niet zo aangemerkt)

In [103]:
np.mean(result_clf>validate["Spam"])*100

0.70280157889669781

In [104]:
np.mean(result_clf<validate["Spam"])*100

1.9736208722441515

In [ ]:
# 0.7% Type I fout, 2.0% Type II. Dus de Naieve Bayes maakt iets meer Type II fouten (mist Spam) dan het random forest model.


## Conclusie Cross-validation

In [126]:
# Beide modellen hebben ongeveer evenveel (weinig) Type I fout
# Naieve Bayes heeft iets hogere Type II fout (laat sommige Spam-mail door)
# Dus: Random forest model selecteren

# Test model

In [ ]:
# Data voorbereiden

In [140]:
clean_test_mails = []
num_test_mails = len(test["text"])
test.index = range(num_test_mails)

for i in xrange( 0, num_test_mails ): 
    clean_test_mails.append( mail_to_words( test["text"][i] ))

In [141]:
test_data_features = vectorizer.transform(clean_test_mails)
test_data_features = test_data_features.toarray()

In [142]:
print test_data_features.shape

(10388L, 5000L)


In [143]:
result_forest2 = forest.predict(test_data_features)

In [144]:
np.mean(result_forest2==test["Spam"])*100

98.411628802464378

In [150]:
100 - np.mean(result_forest2==test["Spam"])*100

1.5883711975356221

In [151]:
np.mean(result_forest2>test["Spam"])*100

0.87601078167115909

In [152]:
np.mean(result_forest2<test["Spam"])*100

0.71236041586445897

In [ ]:
# Lijkt oke te werken :)